## Create Deployment

In this notebook, we'll create a deployment for our recommendation system.

**NOTE Currently the transformer scripts are not implemented.**

In [1]:
import hsml

# connect to Hopsworks Model Registry

conn = hsml.connection()
mr = conn.get_model_registry()

ranking_model = mr.get_best_model("ranking_model", "fscore", "max")

Connected. Call `.close()` to terminate connection gracefully.


In [2]:
import os
import hopsworks

# get Hopsworks Dataset handle

hopsworks_conn = hopsworks.connection()
project = hopsworks_conn.get_project()
dataset_api = project.get_dataset_api()

Connected. Call `.close()` to terminate connection gracefully.


### Ranking Model Deployment

Next, we'll deploy our ranking model. Since it is a CatBoost model we need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it.

In [5]:
%%writefile ranking_predictor.py

import joblib
import os
import numpy as np

class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the path to the model artifact files
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/ranking_model.pkl")

    def predict(self, inputs):
        features = inputs[0].pop("ranking_features")
        article_ids = inputs[0].pop("article_ids")
        
        scores = self.model.predict_proba(features).tolist()
        scores = np.asarray(scores)[:,1].tolist() # get scores of positive class
        
        return { "scores": scores, "article_ids": article_ids }

Overwriting ranking_predictor.py


In [6]:
# copy predictor file into Hopsworks File System

uploaded_file_path = dataset_api.upload("ranking_predictor.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/633 elapsed<00:00 remaining<?

In [11]:
%%writefile ranking_transformer.py

import os
import hsfs
import hsml
import hopsworks
from opensearchpy import OpenSearch

import pandas as pd


class Transformer(object):
    
    def __init__(self):
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature views
        self.articles_fv = self.fs.get_feature_view("articles_fv", 1)
        self.articles_features = [feat.name for feat in self.articles_fv.schema]
        self.customer_fv = self.fs.get_feature_view("customers_fv", 1)
        
        # create opensearch client
        os.environ["ELASTIC_ENDPOINT"] = os.environ["ELASTIC_ENDPOINT"][8:]
        hw_conn = hopsworks.connection()
        project = hw_conn.get_project()
        opensearch_api = project.get_opensearch_api()
        self.os_client = OpenSearch(**opensearch_api.get_default_py_config())
        self.candidate_index = opensearch_api.get_project_index("candidate_index")

        # get ranking model feature names
        mm_conn = hsml.connection()
        mr = mm_conn.get_model_registry()
        model = mr.get_model(os.environ["MODEL_NAME"], os.environ["MODEL_VERSION"])
        input_schema = model.model_schema["input_schema"]["columnar_schema"]
        self.ranking_model_feature_names = [feat["name"] for feat in input_schema]
    
    
    def preprocess(self, inputs):
        inputs = inputs["inputs"][0]
        customer_id = inputs["customer_id"]
                
        # search for candidates
        hits = self.search_candidates(inputs["query_emb"], k=100)
        
        # get already bought items
        already_bought_items_ids = self.fs.sql(
            f"SELECT transactions_2.article_id from transactions_2 WHERE customer_id = '{customer_id}'"
        ).values.reshape(-1).tolist()
        
        # build dfs
        item_id_list = []
        item_emb_list = []
        exclude_set = set(already_bought_items_ids)
        for el in hits:
            item_id = str(el["_id"])
            if item_id in exclude_set:
                continue
            item_emb = el["_source"]["my_vector1"]
            item_id_list.append(item_id)
            item_emb_list.append(item_emb)
        item_id_df = pd.DataFrame({"article_id" : item_id_list})
        item_emb_df = pd.DataFrame(item_emb_list).add_prefix("item_emb_")
        
        # get articles feature vectors
        articles_data = [self.articles_fv.get_feature_vector({"article_id" : article_id}) for article_id in item_id_list]
        articles_df = pd.DataFrame(data=articles_data, columns=self.articles_features)
        ranking_model_inputs = item_id_df.merge(articles_df, on="article_id", how="left")
        
        # add the user features we used with our retrieval model.
        customer_features = self.customer_fv.get_feature_vector({"customer_id": customer_id}) # get customer features
        ranking_model_inputs["age"] = customer_features[1]
        ranking_model_inputs["month_sin"] = inputs["month_sin"]
        ranking_model_inputs["month_cos"] = inputs["month_cos"]
        ranking_model_inputs = ranking_model_inputs[self.ranking_model_feature_names]
        
        return { "inputs" : [{"ranking_features": ranking_model_inputs.values.tolist(), "article_ids": item_id_list}] }

    
    def postprocess(self, outputs):
        preds = outputs["predictions"]
        ranking = list(zip(preds["scores"], preds["article_ids"])) # merge lists
        ranking.sort(reverse=True) # sort by score (descending)
        return { "ranking": ranking }
    

    def search_candidates(self, query_emb, k=100):
        k = 100
        query = {
          "size": k,
          "query": {
            "knn": {
              "my_vector1": {
                "vector": query_emb,
                "k": k
              }
            }
          }
        }
        return self.os_client.search(
            body = query,
            index = self.candidate_index
        )["hits"]["hits"]

Overwriting ranking_transformer.py


In [12]:
# copy transformer file into Hopsworks File System

uploaded_file_path = dataset_api.upload("ranking_transformer.py", "Models", overwrite=True)
transformer_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/4208 elapsed<00:00 remaining<?

This script must be uploaded to the cluster.

With that in place, we can finally deploy our model.

In [9]:
from hsml.transformer import Transformer

ranking_transformer=Transformer(script_file=transformer_script_path)

ranking_deployment = ranking_model.deploy(name="rankingdeployment",
                                          script_file=predictor_script_path,
                                          model_server="PYTHON",
                                          serving_tool="KSERVE",
                                          transformer=ranking_transformer)

In [14]:
# # update deployment with new transformer

# ms = conn.get_model_serving()
# ranking_deployment = ms.get_deployment("rankingdeployment")
# ranking_deployment.script_file=predictor_script_path,
# ranking_deployment.transformer=ranking_transformer
# ranking_deployment.artifact_version="CREATE"
# ranking_deployment.save()

In [15]:
ranking_deployment.start()

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# test ranking deployment

ms = conn.get_model_serving()
ranking_deployment = ms.get_deployment("rankingdeployment")
test_ranking_input = {"inputs": [{"month_sin": 1.2246467991473532e-16,
     "query_emb": [0.0126457736,
      0.511958599,
      -0.0947214961,
      -0.293376535,
      0.468758374,
      0.88662535,
      1.02364039,
      -0.280806333,
      0.228357121,
      0.299074352,
      -0.082454741,
      -0.154759198,
      0.920805335,
      0.0531764328,
      0.234613329,
      1.12010455],
     "month_cos": -1.0,
     "customer_id": "f6e35e1902674780464e8bc0f809cb5ae14883212b4f68b35b31de2facdb846f"}]}

# test ranking
ranking_deployment.predict(test_ranking_input)

{'ranking': [[0.6154721480612619, '736049003'],
  [0.5540913746584363, '675281003'],
  [0.5304851740048736, '699671004'],
  [0.5276218532463308, '662593001'],
  [0.5095340171841166, '551045026'],
  [0.5026730281370363, '690526005'],
  [0.4875691666090818, '684968002'],
  [0.48469640644251866, '733101002'],
  [0.47009852360637683, '664485006'],
  [0.4630649599819785, '792530001'],
  [0.45387771054244613, '685284001'],
  [0.43508248336589944, '636587004'],
  [0.433700596348246, '687524004'],
  [0.4305113042049035, '688873020'],
  [0.4291665002462462, '657497007'],
  [0.4158726009456486, '665477002'],
  [0.40194335335264186, '691446002'],
  [0.39916236382139625, '695601001'],
  [0.37912135024778315, '582894001'],
  [0.37088532655694806, '527687007'],
  [0.37029970448593785, '662344004'],
  [0.3521281188698809, '642047001'],
  [0.343964885720146, '416157003'],
  [0.34029184998960804, '607718001'],
  [0.3394021012791623, '766081001'],
  [0.3337815506689829, '642677001'],
  [0.33347044724992

In [ ]:
# ranking_deployment.stop()

### User Model Deployment

We start by deploying our user/query model.

In [18]:
# get query_model metadata object

user_model = mr.get_model("query_model")

Connected. Call `.close()` to terminate connection gracefully.


In [19]:
%%writefile querymodel_transformer.py

import hsml
import hsfs

import numpy as np
import os

class Transformer(object):
    
    def __init__(self):
        os.environ["ISTIO_ENDPOINT"] = "http://10.132.0.40:32080"
        os.environ["SERVING_API_KEY"] = "p6cHXoRIrqWPd5rC.TJOsBxvTkvq7LXM1zVngiMIxXvrLwkbt7rdTFqUA8I6AEeqnyXcaYkV1hiMO4QMA"
        
        # get feature store handle
        fs_conn = hsfs.connection()
        self.fs = fs_conn.get_feature_store()
        
        # get feature views
        self.customer_fv = self.fs.get_feature_view("customers_fv", 1)
        
        # get model management handle
        mm_conn = hsml.connection() # model management connection
        self.ms = mm_conn.get_model_serving()
        
        # get ranking deployment metadata object
        self.ranking_server = self.ms.get_deployment("rankingdeployment")
        

    def preprocess(self, inputs):
        
        # extract month
        month_of_purchase = inputs.pop("month_of_purchase")
        
        # get customer features
        customer_features = self.customer_fv.get_feature_vector(inputs)
        
        # enrich inputs
        inputs["age"] = customer_features[1]
        inputs["month_sin"], inputs["month_cos"] = self.month_to_unit_circle(month_of_purchase)
        
        return {"instances" : [inputs]}
    
    
    def postprocess(self, outputs):
        # get ordered ranking predictions
        return self.ranking_server.predict({ "inputs": outputs["predictions"] })

    
    def month_to_unit_circle(self, month):
        zero_indexed_month = month - 1
        C = 2*np.pi/12
        month_sin = np.sin(zero_indexed_month*C)
        month_cos = np.cos(zero_indexed_month*C)
        return month_sin, month_cos

Overwriting querymodel_transformer.py


In [21]:
# copy transformer file into Hopsworks File System

uploaded_file_path = dataset_api.upload("querymodel_transformer.py", "Models", overwrite=True)
transformer_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 0.000%|          | 0/1723 elapsed<00:00 remaining<?

In [22]:
from hsml.transformer import Transformer

querymodel_transformer=Transformer(script_file=transformer_script_path)

user_model_deployment = user_model.deploy(name="querymodel",
                                          serving_tool="KSERVE",
                                          transformer=querymodel_transformer)

Using 'TENSORFLOW_SERVING' model server instead


In [ ]:
# update deployment with new transformer
# from hsml.transformer import Transformer

# ms = conn.get_model_serving()
# ranking_deployment = ms.get_deployment("querymodel")
# querymodel_transformer=Transformer(script_file=transformer_script_path)
# user_model_deployment.transformer=querymodel_transformer
# user_model_deployment.artifact_version="CREATE"
# user_model_deployment.save()

At this point, we have registered our deployment. To start it up we need to run:

In [23]:
user_model_deployment.start()

  0%|          | 0/2 [00:00<?, ?it/s]

We can test the deployment by making a prediction on the input example we registered together with the model.

## Get ranking of recommendations by customer

In [24]:
customer_id = "f6e35e1902674780464e8bc0f809cb5ae14883212b4f68b35b31de2facdb846f"
month_of_purchase = 7

query_emb = user_model_deployment.predict({"customer_id": customer_id, "month_of_purchase": month_of_purchase})
query_emb

{'ranking': [[0.6154721480612619, '736049003'],
  [0.5540913746584363, '675281003'],
  [0.5304851740048736, '699671004'],
  [0.5276218532463308, '662593001'],
  [0.5095340171841166, '551045026'],
  [0.5026730281370363, '690526005'],
  [0.4875691666090818, '684968002'],
  [0.48469640644251866, '733101002'],
  [0.47009852360637683, '664485006'],
  [0.4630649599819785, '792530001'],
  [0.45387771054244613, '685284001'],
  [0.43508248336589944, '636587004'],
  [0.433700596348246, '687524004'],
  [0.4305113042049035, '688873020'],
  [0.4291665002462462, '657497007'],
  [0.4158726009456486, '665477002'],
  [0.40194335335264186, '691446002'],
  [0.39916236382139625, '695601001'],
  [0.37912135024778315, '582894001'],
  [0.37088532655694806, '527687007'],
  [0.37029970448593785, '662344004'],
  [0.3521281188698809, '642047001'],
  [0.343964885720146, '416157003'],
  [0.34029184998960804, '607718001'],
  [0.3394021012791623, '766081001'],
  [0.3337815506689829, '642677001'],
  [0.33347044724992

Let's stop the deployment when we're not using it.

In [ ]:
# user_model_deployment.stop()